# Entrenamiento del Modelo Final - Predicción de Precios de Alquiler en AMBA

## Integrantes del Equipo
- Ignacio Bruzone
- Felix Lopez Menardi
- Christian Ijjas

## Fecha
2025

---

Este notebook entrena el modelo final de Random Forest con los mejores hiperparámetros encontrados (en modeling-pipeline.ipynb) y lo guarda para su uso en producción.


In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import json
from datetime import datetime
import os
import warnings

warnings.filterwarnings('ignore')

# Semilla para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Librerías importadas correctamente")


✓ Librerías importadas correctamente


## 1. Carga y Preparación de Datos


In [3]:
# Cargar datos limpios
df = pd.read_csv('output/alquiler_AMBA_clean.csv')

print(f"Dataset cargado: {df.shape[0]:,} registros, {df.shape[1]} columnas")


Dataset cargado: 95,785 registros, 34 columnas


In [4]:
# Identificar variables
target_col = 'precio_pesos_constantes'

# Columnas a excluir del modelado
exclude_cols_base = ['id_grid', 'MesListing', target_col]
exclude_cols = exclude_cols_base.copy()

# Agregar precio_por_m2 a exclude_cols si existe (es derivado del target)
if 'precio_por_m2' in df.columns:
    exclude_cols.append('precio_por_m2')

# Variables numéricas continuas
numeric_cols = ['STotalM2', 'Dormitorios', 'Banos', 'Ambientes', 'Antiguedad', 
                'Cocheras', 'AreaParrillas', 'LONGITUDE', 'LATITUDE', 
                'year', 'densidad_ambientes', 'total_amenities', 'mes']

# Variables binarias (amenities)
binary_cols = ['Amoblado', 'AccesoInternet', 'Gimnasio', 'Laundry', 'Calefaccion',
                'SalonDeUsosMul', 'AireAC', 'Jacuzzi', 'Ascensor', 'Seguridad', 
                'Pileta', 'tiene_cochera']

# Variables categóricas
categorical_cols = ['ITE_ADD_CITY_NAME', 'ITE_ADD_STATE_NAME', 
                    'ITE_ADD_NEIGHBORHOOD_NAME', 'ITE_TIPO_PROD', 'estacion']

# Separar features (X) y target (y)
X = df.drop(columns=[target_col] + exclude_cols)
y = df[target_col]

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")


Features (X): (95785, 30)
Target (y): (95785,)


## 2. Ingeniería de Features


In [5]:
# Aplicar One-Hot Encoding a variables categóricas
categorical_cols_in_X = [col for col in categorical_cols if col in X.columns]
X_encoded = pd.get_dummies(X, columns=categorical_cols_in_X, prefix=categorical_cols_in_X, drop_first=False)

print(f"Shape antes de encoding: {X.shape}")
print(f"Shape después de encoding: {X_encoded.shape}")

# Guardar nombres de columnas finales para uso posterior
feature_names = X_encoded.columns.tolist()
print(f"Total de features después de encoding: {len(feature_names)}")

# Actualizar X con datos procesados
X = X_encoded.copy()

print("✓ Feature engineering completado")


Shape antes de encoding: (95785, 30)
Shape después de encoding: (95785, 562)
Total de features después de encoding: 562
✓ Feature engineering completado


## 3. División de Datos


In [6]:
# Dividir datos en train y validation (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE,
    shuffle=True
)

print("=" * 100)
print("DIVISIÓN DE DATOS")
print("=" * 100)
print(f"\nConjunto de entrenamiento:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")

print(f"\nConjunto de validación:")
print(f"  X_val: {X_val.shape}")
print(f"  y_val: {y_val.shape}")


DIVISIÓN DE DATOS

Conjunto de entrenamiento:
  X_train: (76628, 562)
  y_train: (76628,)

Conjunto de validación:
  X_val: (19157, 562)
  y_val: (19157,)


## 4. Entrenamiento del Modelo Final


In [7]:
# Función para calcular métricas de evaluación
def evaluate_model(y_true, y_pred):
    """Calcula múltiples métricas de evaluación"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE': mape
    }

# Función para imprimir métricas
def print_metrics(metrics, model_name=""):
    """Imprime métricas formateadas"""
    print(f"\n{'='*60}")
    print(f"Métricas - {model_name}")
    print(f"{'='*60}")
    print(f"  RMSE:  ${metrics['RMSE']:,.2f}")
    print(f"  MAE:   ${metrics['MAE']:,.2f}")
    print(f"  R²:    {metrics['R²']:.4f}")
    print(f"  MAPE:  {metrics['MAPE']:.2f}%")
    print(f"{'='*60}")

print("✓ Funciones de evaluación definidas")


✓ Funciones de evaluación definidas


In [8]:
# Entrenar Random Forest con los mejores hiperparámetros encontrados
print("=" * 100)
print("ENTRENANDO: Random Forest Regressor (Mejores Hiperparámetros)")
print("=" * 100)

# Mejores hiperparámetros encontrados en la exploración
best_params = {
    'n_estimators': 200,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_depth': 25
}

print(f"\nHiperparámetros:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

# Crear y entrenar el modelo
rf_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0
)

print("\nEntrenando modelo...")
rf_model.fit(X_train, y_train)
print("✓ Modelo entrenado exitosamente")


ENTRENANDO: Random Forest Regressor (Mejores Hiperparámetros)

Hiperparámetros:
  n_estimators: 200
  min_samples_split: 5
  min_samples_leaf: 2
  max_depth: 25

Entrenando modelo...
✓ Modelo entrenado exitosamente


## 5. Evaluación del Modelo


In [9]:
# Predicciones
y_train_pred = rf_model.predict(X_train)
y_val_pred = rf_model.predict(X_val)

# Evaluación
train_metrics = evaluate_model(y_train, y_train_pred)
val_metrics = evaluate_model(y_val, y_val_pred)

print_metrics(train_metrics, "Random Forest - Train")
print_metrics(val_metrics, "Random Forest - Validation")



Métricas - Random Forest - Train
  RMSE:  $3,810.29
  MAE:   $1,585.11
  R²:    0.9670
  MAPE:  10.44%

Métricas - Random Forest - Validation
  RMSE:  $7,653.68
  MAE:   $3,119.83
  R²:    0.8575
  MAPE:  19.44%


## 6. Guardado del Modelo


In [10]:
# Crear directorio para modelos si no existe
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

# Guardar modelo final
model_filename = os.path.join(models_dir, 'rental_price_model.pkl')
joblib.dump(rf_model, model_filename)
print(f"✓ Modelo guardado: {model_filename}")

# Guardar metadatos del modelo
metadata = {
    'model_name': 'Random Forest',
    'model_type': type(rf_model).__name__,
    'hyperparameters': best_params,
    'features': feature_names,
    'feature_count': len(feature_names),
    'training_samples': len(X_train),
    'validation_samples': len(X_val),
    'metrics': {
        'train': train_metrics,
        'validation': val_metrics
    },
    'random_state': RANDOM_STATE,
    'date_trained': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'target_column': target_col,
    'categorical_columns': categorical_cols_in_X,
    'numeric_columns': numeric_cols,
    'binary_columns': binary_cols
}

metadata_filename = os.path.join(models_dir, 'model_metadata.json')
with open(metadata_filename, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False, default=str)

print(f"✓ Metadatos guardados: {metadata_filename}")

# Guardar información de preprocesamiento (nombres de columnas después de encoding)
preprocessing_info = {
    'feature_names': feature_names,
    'categorical_columns_original': categorical_cols_in_X,
    'numeric_columns': numeric_cols,
    'binary_columns': binary_cols
}

preprocessing_filename = os.path.join(models_dir, 'preprocessing_info.json')
with open(preprocessing_filename, 'w', encoding='utf-8') as f:
    json.dump(preprocessing_info, f, indent=2, ensure_ascii=False)

print(f"✓ Información de preprocesamiento guardada: {preprocessing_filename}")

print("\n" + "="*100)
print("RESUMEN FINAL")
print("="*100)
print(f"\nModelo: Random Forest")
print(f"\nMétricas de validación:")
print_metrics(val_metrics, "Final")
print(f"\nArchivos guardados:")
print(f"  - Modelo: {model_filename}")
print(f"  - Metadatos: {metadata_filename}")
print(f"  - Preprocesamiento: {preprocessing_filename}")
print("\n✓ Entrenamiento completado exitosamente")


✓ Modelo guardado: models/rental_price_model.pkl
✓ Metadatos guardados: models/model_metadata.json
✓ Información de preprocesamiento guardada: models/preprocessing_info.json

RESUMEN FINAL

Modelo: Random Forest

Métricas de validación:

Métricas - Final
  RMSE:  $7,653.68
  MAE:   $3,119.83
  R²:    0.8575
  MAPE:  19.44%

Archivos guardados:
  - Modelo: models/rental_price_model.pkl
  - Metadatos: models/model_metadata.json
  - Preprocesamiento: models/preprocessing_info.json

✓ Entrenamiento completado exitosamente
